In [1]:
import numpy as np 
import pandas as pd
import gensim 
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("~/Desktop/R_js/data/pro_doc_sampled.csv")
display(df.head())
display(df.shape)

,Unnamed: 0,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name
0,0,2.0,1,2017,"hour lunch, friendly co-workers.",cowork friendli hour lunch friendli_cowork hou...,"['cowork', 'friendli', 'hour', 'lunch', 'frien...",9,3.0,2.0,2.0,1.0,3.0,1,Walmart
1,1,3.0,1,2015,"benefits, Medical, dental, myshare, 401k, stocks",benefit dental medic myshar stock,"['benefit', 'dental', 'medic', 'myshar', 'stock']",5,3.0,4.0,4.0,3.0,3.0,1,Walmart
2,2,2.0,0,2012,discount card . schedule 3 on 3 off,card discount schedul discount_card,"['card', 'discount', 'schedul', 'discount_card']",4,1.0,4.0,3.0,1.0,3.0,1,Walmart
3,3,4.0,0,2017,good break lengths and plenty of hours,break hour length plenti plenti_hour,"['break', 'hour', 'length', 'plenti', 'plenti_...",5,4.0,3.0,3.0,4.0,4.0,1,Walmart
4,4,1.0,1,2016,My pay was good and my schedule allowed me to ...,allow job pay schedul pay_schedul,"['allow', 'job', 'pay', 'schedul', 'pay_schedul']",5,3.0,3.0,1.0,1.0,1.0,1,Walmart


(33624, 15)

In [2]:
reviews = df.Review_Text.tolist()
print(reviews[:5])

['hour lunch, friendly co-workers.', 'benefits, Medical, dental, myshare, 401k, stocks', 'discount card . schedule 3 on 3 off', 'good break lengths and plenty of hours', 'My pay was good and my schedule allowed me to work my other jobs']


In [3]:
# Data Clean
import re
import string
import spacy
from nltk.corpus import stopwords 

# stop_words = spacy.lang.en.stop_words.STOP_WORDS
stop_words = list(set(stopwords.words('english')))
stop_words.append("myshare")

def remove_url(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)


def remove_at(text):
    return re.sub('@[^\s]+', '', text)


def remove_hash(text):
    return text.replace("#", "")


def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))


def remove_letter_repitition(text):
    return re.sub(r'\d+','', text)

def remove_numbers(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

def remove_stopwords(text): 
    return " ".join([word for word in text.split() if word not in stop_words])



def clean_data(text):
    text = text.lower()
    text = remove_url(text)
    text = remove_at(text)
    text = remove_hash(text)
    text = remove_punct(text)
    text = remove_letter_repitition(text)
    text = remove_stopwords(text)
    return text


if __name__ == '__main__':
    print(clean_data("What the !! up holy graill aeroplane"))

holy graill aeroplane


In [4]:
tok_reviews = [nltk.word_tokenize(clean_data(review)) for review in reviews]
print(tok_reviews[:5])

[['hour', 'lunch', 'friendly', 'coworkers'], ['benefits', 'medical', 'dental', 'k', 'stocks'], ['discount', 'card', 'schedule'], ['good', 'break', 'lengths', 'plenty', 'hours'], ['pay', 'good', 'schedule', 'allowed', 'work', 'jobs']]


In [5]:
df['Tknz_Texts1'] = tok_reviews

In [6]:
df.head()

,Unnamed: 0,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name,Tknz_Texts1
0,0,2.0,1,2017,"hour lunch, friendly co-workers.",cowork friendli hour lunch friendli_cowork hou...,"['cowork', 'friendli', 'hour', 'lunch', 'frien...",9,3.0,2.0,2.0,1.0,3.0,1,Walmart,"[hour, lunch, friendly, coworkers]"
1,1,3.0,1,2015,"benefits, Medical, dental, myshare, 401k, stocks",benefit dental medic myshar stock,"['benefit', 'dental', 'medic', 'myshar', 'stock']",5,3.0,4.0,4.0,3.0,3.0,1,Walmart,"[benefits, medical, dental, k, stocks]"
2,2,2.0,0,2012,discount card . schedule 3 on 3 off,card discount schedul discount_card,"['card', 'discount', 'schedul', 'discount_card']",4,1.0,4.0,3.0,1.0,3.0,1,Walmart,"[discount, card, schedule]"
3,3,4.0,0,2017,good break lengths and plenty of hours,break hour length plenti plenti_hour,"['break', 'hour', 'length', 'plenti', 'plenti_...",5,4.0,3.0,3.0,4.0,4.0,1,Walmart,"[good, break, lengths, plenty, hours]"
4,4,1.0,1,2016,My pay was good and my schedule allowed me to ...,allow job pay schedul pay_schedul,"['allow', 'job', 'pay', 'schedul', 'pay_schedul']",5,3.0,3.0,1.0,1.0,1.0,1,Walmart,"[pay, good, schedule, allowed, work, jobs]"


In [93]:
model = gensim.models.Word2Vec(tok_reviews, min_count=3, size=250)
model.save("model")

In [94]:
model = gensim.models.Word2Vec.load('model')

In [7]:
model.most_similar('school')

NameError: name 'model' is not defined

In [96]:
print(model['friendly'])
print(model["friendly"].shape)

[-0.16454381 -0.14381346  0.85905075 -0.18687457 -0.31462428  0.41201046
  0.2565868  -0.34588137 -0.08990989  0.49399468 -0.07004149  0.03769401
  0.08687953 -0.2748184   0.04076796 -0.29994145  0.22060853  0.20274244
 -0.12285893 -0.34953722  0.3788669  -0.04427502 -0.19883022  0.07162821
  0.11567874  0.34263685  0.11280752 -0.03610722  0.09207305 -0.0367128
 -0.18689807 -0.3939255   0.4292464  -0.14094041  0.20530114  0.14380713
  0.17209193  0.06982602  0.04982386  0.49353632 -0.502872    0.03118726
 -0.62103194  0.04824969 -0.15384819  0.18784456  0.4458763  -0.3170261
 -0.41488034  0.37679744 -0.02528189 -0.31304494  0.45536405  0.08101735
 -0.05353444 -0.36498114 -0.36253792 -0.23930135  0.24963021  0.34492183
 -0.139573   -0.17025633 -0.32710397 -0.20975256  0.19255427  0.20945005
 -0.296208   -0.02376746  0.24315372 -0.2457317   0.13382919  0.45748872
 -0.5687742   0.07034103 -0.15669794 -0.03732976  0.1539313   0.20996016
  0.3832824   0.05888179  0.05350363  0.30887973  0.0

In [97]:
print(model)

Word2Vec(vocab=3324, size=250, alpha=0.025)


In [116]:
print(model.wv.vocab)

{'hour': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4be0>, 'lunch': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4a90>, 'friendly': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4e48>, 'coworkers': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4e80>, 'benefits': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4eb8>, 'medical': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4c88>, 'dental': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4cf8>, 'k': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4d30>, 'stocks': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4da0>, 'discount': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4dd8>, 'card': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4f28>, 'schedule': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4ef0>, 'good': <gensim.models.keyedvectors.Vocab object at 0x1a29ba4f98>, 'break': <gensim.models.keyedvectors.Vocab object at 0x1a29bc0048>, 'plenty': <gensim.models.keyedvect

In [98]:
# print(vec.wv.vocab)
X = model[model.wv.vocab]
print(model[model.wv.vocab].shape)

(3324, 250)


In [99]:
from sklearn.cluster import KMeans

num_cluster = 8 
kmeans = KMeans(n_clusters=8).fit(X)

In [100]:
clusters = kmeans.labels_.tolist()
# print(clusters)

In [101]:
feat_name = model.wv.vocab
print(feat_name["great"].index)

1


In [102]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [103]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
#         self.dim = len(word2vec.itervalues().next())
#         self.dim = len(word2vec)

#     def fit(self, X, y):
#         return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words], axis=0) for words in X
        ])
    
    
X = MeanEmbeddingVectorizer(w2v).transform(tok_reviews)
print(X.shape)

KeyError: 'lengths'

In [104]:
# KMeans Clustering

from sklearn.cluster import KMeans

num_cluster = 8 
kmeans = KMeans(n_clusters=8).fit(X)

In [105]:
for k in range(8):
    wv = kmeans.cluster_centers_[k]
    print(f"Cluster {k}: ")
    similar_words = model.most_similar(positive=[wv], topn=30)
    print([x[0] for x in similar_words])
    print()

Cluster 0: 
['r', 'desks', 'america', 'nothing', 'volunteering', 'knows', 'wish', 'heat', 'towards', 'gm', 'offerings', 'abilities', 'complete', 'heavy', 'shirt', 'front', 'street', 'national', 'news', 'hotels', 'networks', 'cant', 'console', 'board', 'fund', 'excellence', 'less', 'la', 'contractor', 'en']

Cluster 1: 
['enviroment', 'safe', 'envirnment', 'pleasant', 'enjoyable', 'enviornment', 'understanding', 'caring', 'manager', 'clean', 'workplace', 'managers', 'relaxed', 'comfortable', 'knowledgeable', 'colleagues', 'personable', 'players', 'cooperative', 'helpful', 'collaborative', 'supportive', 'supervisors', 'location', 'members', 'wholesome', 'peers', 'mostly', 'conditions', 'collaborations']

Cluster 2: 
['educate', 'types', 'trades', 'cutting', 'cultures', 'challenges', 'technologies', 'business', 'wise', 'build', 'gained', 'edge', 'things', 'gain', 'improve', 'relationships', 'uses', 'book', 'learned', 'try', 'interact', 'alot', 'supervisory', 'multitask', 'interacting', 'm

In [106]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])
    
X = MeanEmbeddingVectorizer(w2v).transform(tok_reviews)
print(X.shape)

KeyError: 'lengths'

In [107]:
print(X)

[[-2.64312103e-02 -1.05516350e+00  6.32314920e-01 ... -4.19578135e-01
  -9.47186574e-02 -9.00793225e-02]
 [ 1.24626994e-01 -7.37552404e-01  7.97648311e-01 ... -1.97177410e-01
   2.52673984e-01 -3.24843340e-02]
 [-1.64543808e-01 -1.43813461e-01  8.59050751e-01 ... -3.57315361e-01
  -1.08184934e-01 -3.48016471e-01]
 ...
 [ 1.49584606e-01 -1.77112997e-01  2.07249418e-01 ... -9.37697515e-02
   1.88930966e-02 -3.09132729e-02]
 [ 2.31011640e-02 -3.12778465e-02  4.33276109e-02 ... -1.54834120e-02
   9.78356460e-04 -1.26754455e-02]
 [ 1.33182425e-02 -1.75118390e-02  3.15653160e-02 ... -1.65539254e-02
  -6.03470765e-03 -7.06415856e-03]]


In [108]:
# KMeans Clustering

from sklearn.cluster import KMeans

num_cluster = 8 
kmeans = KMeans(n_clusters=8).fit(X)

In [109]:
for k in range(8):
    wv = kmeans.cluster_centers_[k]
    print(f"Cluster {k}: ")
    similar_words = model.most_similar(positive=[wv], topn=30)
    print([x[0] for x in similar_words])
    print()

Cluster 0: 
['r', 'desks', 'volunteering', 'america', 'nothing', 'knows', 'wish', 'towards', 'heat', 'gm', 'offerings', 'abilities', 'shirt', 'heavy', 'front', 'national', 'complete', 'board', 'street', 'networks', 'console', 'news', 'cant', 'hotels', 'contractor', 'excellence', 'fund', 'less', 'la', 'can']

Cluster 1: 
['safe', 'enviroment', 'pleasant', 'colleagues', 'envirnment', 'workplace', 'enviornment', 'enjoyable', 'knowledgeable', 'collaborative', 'supportive', 'conditions', 'caring', 'understanding', 'manager', 'collaborations', 'peers', 'cooperative', 'wholesome', 'managers', 'players', 'managerial', 'personable', 'honest', 'rewarding', 'clean', 'location', 'enviorment', 'generally', 'mostly']

Cluster 2: 
['half', 'unpaid', 'shift', 'four', 'shifts', 'hr', 'per', 'minute', 'weekends', 'two', 'minutes', 'sundays', 'min', 'short', 'fifteen', 'accrue', 'mins', 'thirty', 'one', 'mandatory', 'three', 'early', 'andor', 'seperate', 'months', 'week', 'hrs', 'long', 'according', 'hou

# Trying out LDA on different pre-processed text

In [110]:
data_px = df.Tknz_Texts1.tolist()

In [111]:
# Create Dictionary
id2word = corpora.Dictionary(data_px)
print(len(id2word))
print(id2word)

# Create Corpus
texts = data_px

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])
print(corpus[:5])

9177
Dictionary(9177 unique tokens: ['coworkers', 'friendly', 'hour', 'lunch', 'benefits']...)
[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(11, 1), (13, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [112]:
mallet_path = '~/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word, iterations=500, 
                                            random_seed=42)

In [113]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_px, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.31431422041183954


In [114]:
##mallet model to lda model conversion

optimal_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [115]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint
# %%time
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.093720 -0.137420       1        1  10.047077
3     -0.084768 -0.030520       2        1  10.036695
4     -0.043812  0.040849       3        1  10.021786
7     -0.040290 -0.211077       4        1  10.019655
9      0.006931 -0.295722       5        1  10.018795
5      0.199610 -0.101327       6        1  10.007428
6     -0.249440  0.190179       7        1   9.975926
0      0.279014  0.180607       8        1   9.968948
2      0.277220  0.167663       9        1   9.952362
8     -0.250745  0.196767      10        1   9.951329, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
20    Default  5415.000000           work  5415.000000  30.0000  30.0000
13    Default  5129.000000           good  5129.000000  29.0000  29.0000
4     Default  9034.000000       benefits  9034.000000  28.0000  28.0000
19    Default  5589.000000            pay  5589.000000  27.0000  27.0000
53    Default  7119.000000          great  7119.000000  26.0000  26.0000
77    Default  5452.000000           free  5452.000000  25.0000  25.0000
83    Default  2497.000000           time  2497.000000  24.0000  24.0000
25    Default  3018.000000         people  3018.000000  23.0000  23.0000
3     Default  2220.000000          lunch  2220.000000  22.0000  22.0000
199   Default  2253.000000    environment  2253.000000  21.0000  21.0000
14    Default  2080.000000          hours  2080.000000  20.0000  20.0000
29    Default  1892.000000        lunches  1892.000000  19.0000  19.0000
34    Default  1724.000000       flexible  1724.000000  18.0000  18.0000
41    Default  1647.000000           paid  1647.000000  17.0000  17.0000
0     Default  1540.000000      coworkers  1540.000000  16.0000  16.0000
138   Default  1437.000000         health  1437.000000  15.0000  15.0000
46    Default  1383.000000      excellent  1383.000000  14.0000  14.0000
244   Default  1265.000000           team  1265.000000  13.0000  13.0000
38    Default  1256.000000         breaks  1256.000000  12.0000  12.0000
127   Default  1591.000000     management  1591.000000  11.0000  11.0000
1     Default  1214.000000       friendly  1214.000000  10.0000  10.0000
205   Default  1206.000000           nice  1206.000000   9.0000   9.0000
107   Default  1143.000000        balance  1143.000000   8.0000   8.0000
79    Default  1131.000000           food  1131.000000   7.0000   7.0000
501   Default  1112.000000  opportunities  1112.000000   6.0000   6.0000
403   Default  1103.000000       training  1103.000000   5.0000   5.0000
27    Default  1112.000000            fun  1112.000000   4.0000   4.0000
432   Default  1300.000000        company  1300.000000   3.0000   3.0000
198   Default  1052.000000      employees  1052.000000   2.0000   2.0000
485   Default  1066.000000           home  1066.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
156   Topic10   323.254493     incentives   323.254493   2.3075  -4.0937
39    Topic10   277.541737         minute   277.541737   2.3075  -4.2462
954   Topic10   262.304151       provided   263.369747   2.3034  -4.3027
1613  Topic10   166.525042     membership   166.525042   2.3075  -4.7570
82    Topic10   161.083047         shifts   161.083047   2.3075  -4.7902
2616  Topic10   133.873073     commission   133.873073   2.3075  -4.9753
28    Topic10   120.812285             hr   120.812285   2.3075  -5.0779
227   Topic10   115.370291         awards   115.370291   2.3075  -5.1240
262   Topic10   189.381420            min   190.437868   2.3019  -4.6284
139   Topic10   107.751498          short   107.751498   2.3075  -5.1923
148   Topic10   107.751498      quarterly   107.751498   2.3075  -5.1923
497   Topic10   103.397902          based   103.397902   2.3075  -5.2336
1191  Topic10   102.3095